## [BCE|ECB](https://medium.com/@julienroborel/connecting-to-international-institutions-data-the-sdmx-network-and-the-pandasdmx-library-4ee7c7bd385)
[immagine.png](https://miro.medium.com/v2/resize:fit:720/format:webp/1*B482Dx5GQrDeRZtkzhkEFg.png)
- https://pandasdmx.readthedocs.io/en/v0.9/usage.html
- https://pandasdmx.readthedocs.io/en/v1.0/howto.html

### 1a prova

In [3]:
import sdmx
ecb = sdmx.Request('ecb')
cat_response = ecb.categoryscheme()

Request class will be removed in v3.0; use Client(…)


In [4]:
cat_response

<sdmx.StructureMessage>
  <Header>
    id: 'IREF951111'
    prepared: '2025-07-12T04:38:39+00:00'
    receiver: <Agency not_supplied>
    sender: <Agency Unknown>
    source: 
    test: False
  response: <Response [200]>
  Categorisation (2): 146679AB-6A0F-F0EB-D566-6F63CE655365 7C64141B-4EC...
  CategoryScheme (1): MOBILE_NAVI
  DataflowDefinition (2): EON ILM
  AgencyScheme (1): AGENCIES
  DataStructureDefinition (2): ECB_EON1 ECB_ILM1

In [5]:
sdmx.to_pandas(cat_response.category_scheme.MOBILE_NAVI)

MOBILE_NAVI
01                                  Monetary operations
02             Prices, output, demand and labour market
03                    Monetary and financial statistics
04                                   Euro area accounts
05                                   Government finance
06                  External transactions and positions
07                                       Exchange rates
08    Payments and securities trading, clearing, set...
09                                  Banknotes and Coins
10                  Indicators of Financial Integration
11               Real Time Database (research database)
Name: Economic concepts, dtype: object

In [6]:
cat_response.category_scheme.MOBILE_NAVI

<CategoryScheme ECB:MOBILE_NAVI(1.0) (11 items): Economic concepts>

In [7]:
cat_response

<sdmx.StructureMessage>
  <Header>
    id: 'IREF951111'
    prepared: '2025-07-12T04:38:39+00:00'
    receiver: <Agency not_supplied>
    sender: <Agency Unknown>
    source: 
    test: False
  response: <Response [200]>
  Categorisation (2): 146679AB-6A0F-F0EB-D566-6F63CE655365 7C64141B-4EC...
  CategoryScheme (1): MOBILE_NAVI
  DataflowDefinition (2): EON ILM
  AgencyScheme (1): AGENCIES
  DataStructureDefinition (2): ECB_EON1 ECB_ILM1

In [8]:
flow_msg = ecb.dataflow()
flow_msg

<sdmx.StructureMessage>
  <Header>
    id: 'IREF951212'
    prepared: '2025-07-12T04:38:47+00:00'
    receiver: <Agency not_supplied>
    sender: <Agency Unknown>
    source: 
    test: False
  response: <Response [200]>
  DataflowDefinition (97): AGR AME BKN BLS BNT BOP BSI BSP CAR CBD CBD2...
  DataStructureDefinition (77): ECB_BCS1 ECB_AME1 ECB_BKN1 ECB_BLS1 ECB...

In [9]:
flow_response = ecb.dataflow()
flow_response

<sdmx.StructureMessage>
  <Header>
    id: 'IREF951213'
    prepared: '2025-07-12T04:38:48+00:00'
    receiver: <Agency not_supplied>
    sender: <Agency Unknown>
    source: 
    test: False
  response: <Response [200]>
  DataflowDefinition (97): AGR AME BKN BLS BNT BOP BSI BSP CAR CBD CBD2...
  DataStructureDefinition (77): ECB_BCS1 ECB_AME1 ECB_BKN1 ECB_BLS1 ECB...

In [10]:
exr_flow = ecb.dataflow('EXR')
exr_flow

<sdmx.StructureMessage>
  <Header>
    id: 'IREF951214'
    prepared: '2025-07-12T04:38:48+00:00'
    receiver: <Agency not_supplied>
    sender: <Agency Unknown>
    source: 
    test: False
  response: <Response [200]>
  Categorisation (2): 21e97b57-5950-eaab-eead-1534306b28af 53A341E8-D48...
  CategoryScheme (2): WDC_NAVI WDC_NAVI_OLD
  Codelist (11): CL_COLLECTION CL_CURRENCY CL_DECIMALS CL_EXR_SUFFIX CL...
  ConceptScheme (1): ECB_CONCEPTS
  ContentConstraint (1): EXR_CONSTRAINTS
  DataflowDefinition (1): EXR
  AgencyScheme (1): AGENCIES
  DataStructureDefinition (1): ECB_EXR1

### [2a prova](https://pandasdmx.readthedocs.io/en/v0.9/usage.html) - ECB

In [14]:
from pandasdmx import Request # '*' would do the same
ecb = Request('ECB')
flow_response = ecb.dataflow()

ImportError: cannot import name 'make_generic_validator' from 'pydantic.class_validators' (C:\Users\paolo\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydantic\class_validators.py)

In [15]:
dataflows = sdmx.to_pandas(flow_msg.dataflow)
print(dataflows)

dataflows[dataflows.str.contains("Exchange", case=False)]

exr_msg = ecb.dataflow('EXR')
exr_flow = exr_msg.dataflow.EXR
dsd = exr_flow.structure
print(dsd.dimensions.components)

AGR                                              AGR
AME                                            AMECO
BKN                             Banknotes statistics
BLS                   Bank Lending Survey Statistics
BNT    Shipments of Euro Banknotes Statistics (ESCB)
                           ...                      
SUR                                  Opinion Surveys
TGB                                  Target Balances
TRD                                   External Trade
WTS                                    Trade weights
YC               Financial market data - yield curve
Length: 97, dtype: object
[<Dimension FREQ>, <Dimension CURRENCY>, <Dimension CURRENCY_DENOM>, <Dimension EXR_TYPE>, <Dimension EXR_SUFFIX>, <TimeDimension TIME_PERIOD>]


In [17]:
import pandas as pd

key = '.USD+JPY...'
params = dict(startPeriod = '2016')
ecb = sdmx.Request('ECB')
data_msg = ecb.data('EXR', key=key, params = params)
data = data_msg.data[0]
daily = [s for sk, s in data.series.items() if sk.FREQ == 'D']
cur_df = pd.concat(sdmx.to_pandas(daily)).unstack().T
cur_df.index = pd.DatetimeIndex(cur_df.index,)
cur_df

Request class will be removed in v3.0; use Client(…)


FREQ                 D        
CURRENCY           JPY     USD
CURRENCY_DENOM     EUR     EUR
EXR_TYPE          SP00    SP00
EXR_SUFFIX           A       A
TIME_PERIOD                   
2016-01-04      129.78  1.0898
2016-01-05      127.88  1.0746
2016-01-06      127.19  1.0742
2016-01-07      127.74  1.0868
2016-01-08      128.51  1.0861
...                ...     ...
2025-07-07      170.71  1.1728
2025-07-08      171.70  1.1718
2025-07-09      171.63  1.1698
2025-07-10      171.33  1.1709
2025-07-11      171.81  1.1683

[2439 rows x 2 columns]

### [ESTAT](https://sebastianpaulo.net/blog/working-with-data-from-official-providers-a-brief-tour-of-pandasdmx/)

In [18]:
estat = sdmx.Request('ESTAT')
flow_msg = estat.dataflow()
flow_msg
# https://sebastianpaulo.net/blog/working-with-data-from-official-providers-a-brief-tour-of-pandasdmx/

Request class will be removed in v3.0; use Client(…)


<sdmx.StructureMessage>
  <Header>
    id: 'DF1752267615'
    prepared: '2025-07-11T23:00:15.070000+02:00'
    sender: <Agency ESTAT>
    source: 
    test: False
  response: <Response [200]>
  DataflowDefinition (7950): LFST_R_LFE2EMP LFST_R_LFE2EMPRC LFST_R_LFE...
  DataStructureDefinition (7501): LFST_R_LFE2EMP LFST_R_LFE2EMPRC LFST_...

In [19]:
dataflows = sdmx.to_pandas(flow_msg.dataflow)
type(dataflows)

pandas.core.series.Series

In [20]:
dataflows.head()

LFST_R_LFE2EMP        Employment by sex, age and NUTS 2 region (1 000)
LFST_R_LFE2EMPRC     Employment rates by sex, age, educational atta...
LFST_R_LFE2EMPRT     Employment rates by sex, age and NUTS 2 region...
LFST_R_LFE2EMPRTN    Employment rates by sex, age, educational atta...
LFST_R_LFE2EN1       Employment by sex, age, economic activity and ...
dtype: object

In [21]:
internet_use_flows = dataflows[dataflows.str.contains('Individuals using the internet', case=False)]
internet_use_flows

TIN00094    Individuals using the internet for sending/rec...
TIN00095    Individuals using the internet for finding inf...
TIN00096    Individuals using the internet for buying good...
TIN00098    Individuals using the internet for selling goo...
TIN00099    Individuals using the internet for internet ba...
TIN00101    Individuals using the internet for seeking hea...
TIN00102    Individuals using the internet for looking for...
TIN00103    Individuals using the internet for doing an on...
TIN00127    Individuals using the internet for participati...
TIN00129    Individuals using the internet for taking part...
dtype: object

In [22]:
tin_msg = estat.dataflow('TIN00099')
tin_msg

<sdmx.StructureMessage>
  <Header>
    id: 'DF1750929217'
    prepared: '2025-06-26T09:13:37.467000+00:00'
    sender: <Agency ESTAT>
    source: 
    test: False
  response: <Response [200]>
  Codelist (7): FREQ INDIC_IS UNIT IND_TYPE GEO OBS_FLAG CONF_STATUS
  ConceptScheme (1): TIN00099
  DataflowDefinition (1): TIN00099
  DataStructureDefinition (1): TIN00099

In [23]:
tin_flow = tin_msg.dataflow.TIN00099
dsd = tin_flow.structure
dsd

<DataStructureDefinition ESTAT:TIN00099(47.0): TIN00099 data structure>

In [24]:
dsd.dimensions.components

[<Dimension freq>,
 <Dimension indic_is>,
 <Dimension unit>,
 <Dimension ind_type>,
 <Dimension geo>,
 <TimeDimension TIME_PERIOD>]

In [25]:
cl_freq = dsd.dimensions.get('freq').local_representation.enumerated
cl_freq

<Codelist ESTAT:FREQ(3.7) (11 items): Time frequency>

In [26]:
dsd.attributes.components

[DataAttribute(annotations=[], id='OBS_FLAG', uri=None, urn='urn:sdmx:org.sdmx.infomodel.datastructure.DataAttribute=ESTAT:TIN00099(47.0).OBS_FLAG', concept_identity=<Concept OBS_FLAG: Observation status (Flag) V2 structure>, local_representation=<Representation: OBS_FLAG, []>, related_to=<sdmx.model.v21.PrimaryMeasureRelationship object at 0x000002312403B910>, usage_status=<UsageStatus.conditional: 2>, concept_role=None),
 DataAttribute(annotations=[], id='CONF_STATUS', uri=None, urn='urn:sdmx:org.sdmx.infomodel.datastructure.DataAttribute=ESTAT:TIN00099(47.0).CONF_STATUS', concept_identity=<Concept CONF_STATUS: Confidentiality status (flag)>, local_representation=<Representation: CONF_STATUS, []>, related_to=<sdmx.model.v21.PrimaryMeasureRelationship object at 0x00000231240A85D0>, usage_status=<UsageStatus.conditional: 2>, concept_role=None)]

In [27]:
dsd.measures.components

[<PrimaryMeasure OBS_VALUE>]